## Import modules


In [ ]:
from spezi_data_pipeline.data_access.firebase_fhir_data_access import FirebaseFHIRAccess, FHIRResourceType
from spezi_data_pipeline.data_flattening.fhir_resources_flattener import extract_questionnaire_mappings, flatten_fhir_resources, FHIRDataFrame, QuestionnaireResponseFlattener
from spezi_data_pipeline.data_processing.data_processor import FHIRDataProcessor
from spezi_data_pipeline.data_processing.questionnaire_processor import calculate_risk_score
from spezi_data_pipeline.data_exploration.data_explorer import DataExplorer, visualizer_factory, explore_total_records_number
from spezi_data_pipeline.data_export.data_exporter import DataExporter

## Define credential files

In [ ]:
# Define your Firebase project ID
project_id = "spezi-data-pipeline"  # Replace with your Firebase project ID

# Define the service account key file
service_account_key_file = "path_to_service_account_key_file.json"  # Replace with your service account key file

# Define the collection name where your FHIR observations are stored and the input code if filtering is needed
collection_name = "users"
subcollection_name = "QuestionnaireResponse"

# Define the survey path for creating questionnaire mappings
questionnaire_resource_path = "path_to_the_questionnaire_resource_file.json"

## Initialize the FirebaseFHIRAccess class using your Firebase credentials

In [ ]:
# Initialize the FirebaseFHIRAccess class using your Firebase credentials
firebase_access = FirebaseFHIRAccess(project_id, service_account_key_file)
firebase_access.connect()

fhir_questionnaires = firebase_access.fetch_data(collection_name, subcollection_name)

flattened_fhir_dataframe = flatten_fhir_resources(fhir_questionnaires, questionnaire_path)

## Export data

In [ ]:
exporter = DataExporter(flattened_fhir_dataframe)
exporter.export_to_csv("survey_data.csv") 

## Calculate risk score

In [ ]:
questionnaire_title = "questionnaire_title"
flattened_fhir_dataframe_with_score = calculate_risk_score(flattened_fhir_dataframe, questionnaire_title)

In [ ]:
explorer = visualizer_factory(flattened_fhir_dataframe_with_score, questionnaire_title="PHQ-9")
# explorer.set_user_ids(["User1", "User2"])
# explorer.set_date_range("2023-01-13", "2023-02-16")
fig = explorer.create_score_plot(flattened_fhir_dataframe_with_score)